<a href="https://colab.research.google.com/github/SiaZin/2024-Ukraine-massive-shelling-analysis/blob/main/Massive_shelling_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Отримання даних про обстріли з телеграм-каналу


<p>The default order is from newest to oldest</p>
<p>Message object;   class telethon.tl.custom.message.Message
<p>message.date is a datetime object
<p>message.message is a str
<p>Лише ті message, які мають фото  і які лежать у проміжку 01.01.2024-31.12.2024

In [ ]:
!pip install telethon

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 12.6 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=d4702ccd1280bf55c568ef162dbaacef809b19a18996b8af151b6b6ae9de9884
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [ ]:
!pip install nest_asyncio

In [ ]:
#await client.disconnect()

In [ ]:
import asyncio
from telethon.sync import TelegramClient
from telethon.tl.types import InputMessagesFilterPhotos
#from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.functions.messages import SearchRequest

import nest_asyncio
nest_asyncio.apply()

import time
import datetime

In [ ]:
import pandas as pd

# Глобальна змінна для збереження DataFrame
messages_df = None

In [ ]:
'''
api_id = ***
api_hash = '***'
'''
session_name = 'telegram_session'

In [ ]:
BEGIN_DATE = datetime.datetime(2024, 1, 1, 0, 0, 0)
END_DATE = datetime.datetime(2024, 12, 31,23,59,0)

In [ ]:
# Функція для отримання повідомлень
async def fetch_messages():
    async with TelegramClient(session_name, api_id, api_hash) as client:
        channel_username = 'kpszsu'  # Ім'я каналу
        channel = await client.get_entity(channel_username)  # Отримуємо канал

        messages = []

        # client.iter_messages() повертає екземпляри класу Message
        async for message in client.iter_messages(channel, limit=4000, filter = InputMessagesFilterPhotos, wait_time=3, offset_date=END_DATE):
            if any(word.lower() in (message.message or '').lower() for word in ['стратегічна авіація', 'Ту-95', 'Ту-22М3', 'комбінований удар','Ту-95мс']):
                messages.append({
                    'date': message.date,
                    'text': message.message
                })
                #time.sleep(1)
        return messages

# Виклик асинхронної функції await fetch_messages() або asyncio.run(fetch_messages())

async def main():
    messages = await fetch_messages()

    global messages_df
    messages_df = pd.DataFrame(messages)  # Створюємо DataFrame
    print(messages_df.head())  # Виводимо перші кілька рядків для перевірки

'''
    for message in messages:
        print(message)
'''
# Запуск основної асинхронної функції. Точка входу
asyncio.run(main())

                       date                                               text
0 2024-12-31 10:35:21+00:00  ⚡️ ЗБИТО 6 РАКЕТ ТА 16 ВОРОЖИХ БПЛА, ЩЕ 24 БЕЗ...
1 2024-12-25 10:43:45+00:00  ⚡️ ЗБИТО 113 ПОВІТРЯНИХ ЦІЛЕЙ \n➖➖➖➖➖➖➖➖➖➖\nУ ...
2 2024-12-13 09:55:13+00:00  ⚡️ ЗБИТО 81 РАКЕТУ ТА 80 ВОРОЖИХ БПЛА, ЩЕ 105 ...
3 2024-11-28 08:52:47+00:00  ⚡️ ЗБИТО 76 КРИЛАТИХ РАКЕТ, 3 КЕРОВАНІ АВІАЦІЙ...
4 2024-11-21 07:23:23+00:00  ⚡️ ЗБИТО ШІСТЬ КРИЛАТИХ РАКЕТ\n➖➖➖➖➖➖➖➖➖➖\nУра...


In [ ]:
messages_df

,date,text
0,2024-12-31 10:35:21+00:00,"⚡️ ЗБИТО 6 РАКЕТ ТА 16 ВОРОЖИХ БПЛА, ЩЕ 24 БЕЗ..."
1,2024-12-25 10:43:45+00:00,⚡️ ЗБИТО 113 ПОВІТРЯНИХ ЦІЛЕЙ \n➖➖➖➖➖➖➖➖➖➖\nУ ...
2,2024-12-13 09:55:13+00:00,"⚡️ ЗБИТО 81 РАКЕТУ ТА 80 ВОРОЖИХ БПЛА, ЩЕ 105 ..."
3,2024-11-28 08:52:47+00:00,"⚡️ ЗБИТО 76 КРИЛАТИХ РАКЕТ, 3 КЕРОВАНІ АВІАЦІЙ..."
4,2024-11-21 07:23:23+00:00,⚡️ ЗБИТО ШІСТЬ КРИЛАТИХ РАКЕТ\n➖➖➖➖➖➖➖➖➖➖\nУра...
...,...,...
72,2022-06-05 09:30:55+00:00,🇺🇦 Повітряне командування «Центр» інформує:\n⚡...
73,2022-05-25 20:22:03+00:00,УКРАЇНСЬКІ ВИНИЩУВАЧІ ЗБИВАЮТЬ ДВІ КРИЛАТІ РАК...
74,2022-05-03 21:01:56+00:00,Увечері 3 травня – повітряна тривога по всій т...
75,2022-04-25 08:58:09+00:00,🇺🇦Повітряне командування «Захід» інформує:\n\n...


In [ ]:
print("Остаточний DataFrame:")
print(messages_df.info())

Остаточний DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   date    77 non-null     datetime64[ns, UTC]
 1   text    77 non-null     object             
dtypes: datetime64[ns, UTC](1), object(1)
memory usage: 1.3+ KB
None


In [ ]:
messages_df.to_csv('Shelling_raw_data.csv')    #saves in UTF-8, cause problems with Excel for ukrainian